In [50]:
import pandas as pd
from assign_unique_ids_functions import *
import inspect
import itertools
import numpy as np

In [2]:
def intersect(a, b):
     return list(set(a) & set(b))
def setdiff(a,b):
    return list(set(a) - set(b))
def union(a,b):
    return list(set(a) | set(b))

In [119]:
asd = pd.read_csv("input/all-sworn_demographics.csv.gz")
asud = pd.read_csv("input/all-sworn-units_demographics.csv.gz")
print(asd.shape)
print(asud.shape)

(32006, 9)
(32006, 10)


In [121]:
asd[asd['Last.Name'].str.contains('JUR')].sort_values('First.Name')

,all_sworn_ID,First.Name,Last.Name,Suffix.Name,Appointed.Date,Birth.Year,Gender,Race,Middle.Initial
14049,31996,AUDREY,JURCZYKOWSKI,NaN,1999-05-10,1970,FEMALE,WHITE,A
14058,6213,DENNIS,JURKOWSKI,NaN,1967-11-13,1946,MALE,WHITE,A
14053,9357,GEO,JURGENSEN,NaN,1953-09-21,1920,MALE,WHITE,O
14055,9500,GEORGE,JURICH,NaN,1958-05-01,1932,MALE,WHITE,P
14059,11761,JAMES,JURTA,NaN,1968-02-26,1947,MALE,WHITE,J
14051,14038,JOHN,JUREK,NaN,1995-10-23,1973,MALE,WHITE,J
14056,14039,JOHN,JURISS,NaN,1956-05-16,1931,MALE,WHITE,J
14047,19105,MARINA,JURASSI,NaN,2006-08-28,1979,FEMALE,WHITE,L
7040,21704,NEBOJSA,DJURDJEVIC,NaN,2014-02-18,1984,MALE,WHITE,NaN
14052,24449,RICHARD,JUREK,NaN,1962-03-12,1938,MALE,WHITE,R


In [4]:
def take_first_four(x):
    return x[:4]


def BY_to_CA(x):
    return 2016 - x


def add_columns(df, add_cols = ["F4FN", "F4LN", "Current.Age", "BY_to_CA"]):
    if "F4FN" in add_cols and "First.Name" in df.columns:
            df['F4FN'] = df['First.Name'].map(take_first_four)
    if "F4LN" in add_cols and 'Last.Name' in df.columns:
            df['F4LN'] = df['Last.Name'].map(take_first_four)
    if "Current.Age" in add_cols and "Current.Age" in df.columns:
        df['Current.Age.p1'] = df['Current.Age']
        df['Current.Age.m1'] = df['Current.Age']
    if "BY_to_CA" in add_cols and "Birth.Year" in df.columns:
        by_to_ca = lambda x: 2016 - x
        df['Current.Age.p1'] = df['Birth.Year'].map(BY_to_CA)
        df['Current.Age.m1'] = df['Birth.Year'].map(BY_to_CA) - 1  
    return df

In [5]:
def generate_on_lists(data_cols, base_lists):
    merge_list = []
    
    for col_list in base_lists:
        if intersect(col_list, data_cols):
            ml = intersect(col_list, data_cols)
            if '' in col_list:
                ml.append('')
            merge_list.append(sorted(ml, reverse=True))
            
    merge_list = list(itertools.product(*reversed(merge_list)))
    merge_list = [[i for i in ml if i != ''] for ml in merge_list]
    
    return merge_list

In [6]:
def loop_merge(df1, df2, on_lists, keep_columns, return_unmatched = True):
    dfm = pd.DataFrame(columns = keep_columns + ['Match'])
    for mc in on_lists:
        df1t = remove_duplicates(df1[keep_columns[:1] + mc], mc)
        df2t = remove_duplicates(df2[keep_columns[1:] + mc], mc)
        dfmt = df1t.merge(df2t, on=mc, how='inner')
        if dfmt.shape[0] > 0:
            print('******')
            print(mc)
            print(dfmt.shape[0])
            print('******')
            dfmt['Match'] = '-'.join(mc)
            dfm = dfm.append(dfmt[keep_columns + ['Match']].reset_index(drop=True))
            df1 = df1.loc[~df1[keep_columns[0]].isin(dfm[keep_columns[0]])]
            df2 = df2.loc[~df2[keep_columns[1]].isin(dfm[keep_columns[1]])]
    print(dfm.shape[0], df1.shape[0], df2.shape[0])
    if return_unmatched:
        return (dfm.reset_index(drop=True), df1, df2)
    else:
        return dfm.reset_index(drop=True)

In [7]:
def merge_datasets(df1, df2, keep_columns,
                   custom_matches = [], return_unmatched = True, name_changes=True):
    df1 = df1.dropna(axis=1, how='all')
    df2 = df2.dropna(axis=1, how='all')
    
    if "Birth.Year" not in intersect(df1.columns, df2.columns):
        add_cols = ["F4FN", "F4LN", "BY_to_CA", "Current.Age"]
    else:
        add_cols = ["F4FN", "F4LN"]
        
    df1 = add_columns(df1, add_cols)
    df2 = add_columns(df2, add_cols)
    
    
    cols = intersect(df1.columns, df2.columns)
    
    df1 = df1[[col for col in df1.columns
               if col in cols or col == keep_columns[0]]]
    df2 = df2[[col for col in df2.columns
               if col in cols or col == keep_columns[1]]]
    
    base_lists = [
        ['Current.Star', 'Star1', 'Star2', 'Star3', 'Star4', 'Star5','Star6', 'Star7', 'Star8', 'Star9', 'Star10'],
        ['First.Name', 'F4FN'], ['Last.Name', 'F4LN'], ['Appointed.Date'],
        ['Birth.Year', 'Current.Age', 'Current.Age.p1', 'Current.Age.m1', ''],
        ['Middle.Initial', ''], ['Gender', ''], ['Race', ''], ['Suffix.Name', '']
    ]
    
    on_lists = generate_on_lists(cols, base_lists)
    
    if custom_matches:
        on_lists.append(custom_matches)
    
    if name_changes:
        nc_lists = generate_on_lists(cols, [ml for ml in base_lists if "Last.Name" not in ml])
        nc_lists = [nc_list for nc_list in nc_lists if len(nc_list) > 3]
        on_lists.extend(nc_lists)
    
    merged_data = loop_merge(df1, df2, on_lists=on_lists, keep_columns=keep_columns, return_unmatched=return_unmatched)
        
    return(merged_data)

In [88]:
def append_to_reference(df1, df2, keep_columns, custom_matches = [], return_unmatched=False, name_changes=True):
    ml = merge_datasets(df1, df2, keep_columns=keep_columns, 
                        custom_matches=custom_matches, name_changes=name_changes)

    ref = pd.concat([ml[0][keep_columns],
                    ml[1][keep_columns[0]],
                    ml[2][keep_columns[1]]]).reset_index(drop=True)
    
    if "UID" not in ref.columns:
        ref.insert(0, 'UID', ref.index + 1)
    
    df1 = df1.merge(ref, on=keep_columns[0], how='left')
    df2 = df2.merge(ref, on=keep_columns[1], how='left')

    ref = pd.concat([df1, df2]).reset_index(drop=True)
    if return_unmatched:
        return (ref, ml[1], ml[2])
    else:
        return ref

In [29]:
t1 = append_to_reference(asd, asud, ["all_sworn_ID", "all_sworn_units_ID"])

******
['Suffix.Name', 'Race', 'Gender', 'Middle.Initial', 'Birth.Year', 'Appointed.Date', 'Last.Name', 'First.Name']
32006
******


/Users/invinst/anaconda3/lib/python3.6/site-packages/pandas/core/reshape/merge.py:1457: RuntimeWarning: divide by zero encountered in long_scalars
  stride //= shape[i]


32006 0 0


/Users/invinst/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/api.py:77: RuntimeWarning: '>' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  result = result.union(other)


In [111]:
def aggregate_data2(df, uid, id_cols=[],
                   mode_cols=[], max_cols=[],
                   current_cols=[], time_col=""):
    from statistics import mode
    import numpy as np
    uid_col = [uid]
    agg_df = df[uid_col + id_cols].drop_duplicates()
    agg_df.reset_index(drop=True, inplace=True)
    
    agg_cols = max_cols + mode_cols
    for col in agg_cols:
        dd = df[[uid, col]].drop_duplicates().dropna()
        kd = keep_duplicates(dd, uid)
        if not kd.empty:
            groups = kd.groupby(uid, as_index=False)
            if col in max_cols:
                groups = groups.agg(np.nanmax)
            if col in mode_cols:
                groups = groups.agg(mode)                
            agg_df = agg_df.merge(groups, on=uid, how='left')
        else:
            agg_df = agg_df.merge(dd, on=uid, how='left')
    if current_cols and time_col:
        df[time_col] = pd.to_datetime(df[time_col])
        agg_df = agg_df.merge(
                    order_aggregate(
                        df[uid_col + [time_col] + current_cols],
                        uid_col, current_cols, [time_col]),
                    on=uid, how='left')
        agg_df.rename(columns=dict(
                            zip(current_cols,
                                ["Current." + tc for tc in current_cols])),
                      inplace=True)

    return agg_df

In [113]:
reft = ref.sort_values("UID").iloc[0:5,:]
profile_cols = ["First.Name", "Last.Name", "Middle.Initial", "Suffix.Name",
                "Appointed.Date", "Birth.Year", "Current.Unit", "Race", "Gender"]
mode_cols = profile_cols
aggregate_data2(reft, "UID", mode_cols = mode_cols)

Empty DataFrame
Columns: [UID, First.Name]
Index: []
Empty DataFrame
Columns: [UID, Last.Name]
Index: []
Empty DataFrame
Columns: [UID, Middle.Initial]
Index: []
Empty DataFrame
Columns: [UID, Suffix.Name]
Index: []
Empty DataFrame
Columns: [UID, Appointed.Date]
Index: []
Empty DataFrame
Columns: [UID, Birth.Year]
Index: []
Empty DataFrame
Columns: [UID, Current.Unit]
Index: []
Empty DataFrame
Columns: [UID, Race]
Index: []
Empty DataFrame
Columns: [UID, Gender]
Index: []


,UID,First.Name,Last.Name,Middle.Initial,Suffix.Name,Appointed.Date,Birth.Year,Current.Unit,Race,Gender
0,1,RICHARD,EVERLY,A,II,1981-01-19,1948,51.0,BLACK,MALE
1,2,ERNEST,HARRIS,B,II,1967-11-13,1947,21.0,BLACK,MALE
2,3,EDGAR,ANDERSON,B,II,1973-03-19,1948,3.0,BLACK,MALE


In [115]:
ref = t1
profile_cols = ["First.Name", "Last.Name", "Middle.Initial", "Suffix.Name",
                "Appointed.Date", "Birth.Year", "Current.Unit", "Race", "Gender"]
mode_cols = profile_cols
profiles = aggregate_data2(ref, "UID", mode_cols = mode_cols)

Empty DataFrame
Columns: [UID, First.Name]
Index: []
Empty DataFrame
Columns: [UID, Last.Name]
Index: []
Empty DataFrame
Columns: [UID, Middle.Initial]
Index: []
Empty DataFrame
Columns: [UID, Suffix.Name]
Index: []
Empty DataFrame
Columns: [UID, Appointed.Date]
Index: []
Empty DataFrame
Columns: [UID, Birth.Year]
Index: []
Empty DataFrame
Columns: [UID, Current.Unit]
Index: []
Empty DataFrame
Columns: [UID, Race]
Index: []
Empty DataFrame
Columns: [UID, Gender]
Index: []


In [102]:
amd = pd.read_csv("input/all-members_demographics.csv.gz")
amd.head()

,all_members_ID,First.Name,Last.Name,Middle.Initial,Suffix.Name,Appointed.Date,Birth.Year,Gender,Race
0,1,JEFFERY,AARON,M,NaN,2005-09-26,1971,MALE,WHITE
1,2,KARINA,AARON,NaN,NaN,2005-09-26,1980,FEMALE,HISPANIC
2,3,DANIEL,ABATE,P,NaN,1970-06-15,1942,MALE,WHITE
3,4,ANTHONY,ABBATE,G,NaN,1994-12-05,1968,MALE,WHITE
4,5,CARMEL,ABBATE,G,NaN,1969-01-06,1942,MALE,WHITE


In [89]:
t2 = append_to_reference(profiles, k, ["UID", "all_members_ID"],return_unmatched=True)

******
['Suffix.Name', 'Race', 'Gender', 'Middle.Initial', 'Birth.Year', 'Appointed.Date', 'Last.Name', 'First.Name']
31911
******
******
['Suffix.Name', 'Race', 'Gender', 'Middle.Initial', 'Birth.Year', 'Appointed.Date', 'F4LN', 'First.Name']
3
******
******
['Suffix.Name', 'Race', 'Gender', 'Middle.Initial', 'Appointed.Date', 'Last.Name', 'First.Name']
1
******
******
['Suffix.Name', 'Gender', 'Middle.Initial', 'Birth.Year', 'Appointed.Date', 'Last.Name', 'First.Name']
1
******
******
['Suffix.Name', 'Race', 'Gender', 'Middle.Initial', 'Birth.Year', 'Appointed.Date', 'First.Name']
8
******
31924 82 4


/Users/invinst/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/api.py:77: RuntimeWarning: '>' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  result = result.union(other)


In [90]:
t2[2]

,all_members_ID,First.Name,Last.Name,Middle.Initial,Suffix.Name,Appointed.Date,Birth.Year,Gender,Race,F4FN,F4LN
1942,1943,NACALA,BEY,M,NaN,1989-12-26,1964,FEMALE,BLACK,NACA,BEY
14016,14017,AUDREY,JURCZYKOWSKI,A,NaN,1999-05-10,1970,FEMALE,WHITE,AUDR,JURC
20291,20292,STEPHEN,MYTHEN,C,NaN,2000-09-11,1962,MALE,WHITE,STEP,MYTH
29842,29843,ERICK,VONKONDRAT,M,NaN,1999-05-10,1976,MALE,WHITE,ERIC,VONK


In [ ]:
t2[0][t2[0]['all_members_ID'] == 14018]

In [99]:
amd[amd['Last.Name'].str.contains('JUR')].sort_values('First.Name')

,all_members_ID,First.Name,Last.Name,Middle.Initial,Suffix.Name,Appointed.Date,Birth.Year,Gender,Race
14016,14017,AUDREY,JURCZYKOWSKI,A,NaN,1999-05-10,1970,FEMALE,WHITE
14017,14018,AUDREY,JURCZYKOWSKI,NaN,NaN,1999-05-10,1970,FEMALE,WHITE
14026,14027,DENNIS,JURKOWSKI,A,NaN,1967-11-13,1946,MALE,WHITE
14021,14022,GEO,JURGENSEN,O,NaN,1953-09-21,1920,MALE,WHITE
14023,14024,GEORGE,JURICH,P,NaN,1958-05-01,1932,MALE,WHITE
14027,14028,JAMES,JURTA,J,NaN,1968-02-26,1947,MALE,WHITE
14019,14020,JOHN,JUREK,J,NaN,1995-10-23,1973,MALE,WHITE
14024,14025,JOHN,JURISS,J,NaN,1956-05-16,1931,MALE,WHITE
14014,14015,MARINA,JURASSI,L,NaN,2006-08-28,1979,FEMALE,WHITE
7026,7027,NEBOJSA,DJURDJEVIC,NaN,NaN,2014-02-18,1984,MALE,WHITE


In [13]:
groups = t1.groupby('UID', as_index=False)
t12 = groups.agg('max')

In [17]:
t12.tail()

,UID,Appointed.Date,Birth.Year,Current.Unit,First.Name,Gender,Last.Name,Middle.Initial,Race,Suffix.Name,all_sworn_ID,all_sworn_units_ID
64007,64008,2016-02-29,1980,44.0,KATARZYNA,FEMALE,ZYLINSKA,,WHITE,NaN,16450,16450
64008,64009,1986-10-13,1956,650.0,DEBRA,FEMALE,ZYLVITIS,A,WHITE,NaN,6037,6037
64009,64010,2002-12-02,1974,14.0,MARK,MALE,ZYMANTAS,E,WHITE,NaN,19389,19389
64010,64011,1996-12-02,1970,20.0,CARLO,MALE,ZYRKOWSKI,E,WHITE,NaN,3057,3057
64011,64012,1955-02-08,1932,631.0,STANLEY,MALE,ZYSKOWSKI,H,WHITE,NaN,27928,27928


In [287]:
t1.tail(20)

,UID,Appointed.Date,Birth.Year,Current.Unit,First.Name,Gender,Last.Name,Race,Suffix.Name,all_sworn_ID,all_sworn_units_ID
63992,63993,1986-10-13,1956,650.0,DEBRA,FEMALE,ZYLVITIS,WHITE,NaN,6037,6037
63993,63994,2002-12-02,1974,14.0,MARK,MALE,ZYMANTAS,WHITE,NaN,19389,19389
63994,63995,1996-12-02,1970,20.0,CARLO,MALE,ZYRKOWSKI,WHITE,NaN,3057,3057
63995,63996,1955-02-08,1932,631.0,STANLEY,MALE,ZYSKOWSKI,WHITE,NaN,27928,27928
63996,63997,1970-01-19,1948,NaN,ROBERT,MALE,ANDERSON,WHITE,NaN,32000,NaN
63997,63998,1970-01-19,1948,NaN,ROBERT,MALE,ANDERSON,WHITE,NaN,32001,NaN
63998,63999,2009-12-16,1983,NaN,JAMES,MALE,BANSLEY,WHITE,NaN,31998,NaN
63999,64000,2009-12-16,1983,NaN,JAMES,MALE,BANSLEY,WHITE,NaN,31999,NaN
64000,64001,1950-06-03,1926,NaN,ROBERT,MALE,BARRETT,WHITE,NaN,32002,NaN
64001,64002,1950-06-03,1926,NaN,ROBERT,MALE,BARRETT,WHITE,NaN,32003,NaN


In [239]:
uhd = pd.read_csv("input/unit-history_demographics.csv.gz")

In [249]:
uhd.columns

Index(['unit_history_ID', 'First.Name', 'Last.Name', 'Middle.Initial',
       'Suffix.Name', 'Appointed.Date', 'Current.Age', 'Gender', 'Race',
       'Star1', 'Star2', 'Star3', 'Star4', 'Star5', 'Star6', 'Star7', 'Star8',
       'Star9', 'Star10', 'Current.Unit'],
      dtype='object')

In [250]:
asd.columns

Index(['all_sworn_ID', 'First.Name', 'Last.Name', 'Middle.Initial',
       'Suffix.Name', 'Appointed.Date', 'Birth.Year', 'Gender', 'Race'],
      dtype='object')

In [252]:
t2 = merge_datasets(asd, uhd, , base_lists, name_changes=True)

******
['Suffix.Name', 'Race', 'Gender', 'Middle.Initial', 'Current.Age.p1', 'Appointed.Date', 'Last.Name', 'First.Name']
1311
******
******
['Suffix.Name', 'Race', 'Gender', 'Middle.Initial', 'Current.Age.m1', 'Appointed.Date', 'Last.Name', 'First.Name']
4287
******
******
['Suffix.Name', 'Race', 'Gender', 'Middle.Initial', 'Current.Age.m1', 'Appointed.Date', 'F4LN', 'First.Name']
2
******
******
['Suffix.Name', 'Race', 'Gender', 'Middle.Initial', 'Appointed.Date', 'Last.Name', 'First.Name']
1
******
******
['Suffix.Name', 'Race', 'Gender', 'Current.Age.p1', 'Appointed.Date', 'Last.Name', 'First.Name']
6014
******
******
['Suffix.Name', 'Race', 'Gender', 'Current.Age.m1', 'Appointed.Date', 'Last.Name', 'First.Name']
19934
******
******
['Suffix.Name', 'Race', 'Gender', 'Current.Age.m1', 'Appointed.Date', 'Last.Name', 'F4FN']
1
******
******
['Suffix.Name', 'Race', 'Gender', 'Current.Age.m1', 'Appointed.Date', 'F4LN', 'First.Name']
4
******
******
['Suffix.Name', 'Race', 'Gender', 'App

In [259]:
t3 = merge_datasets(asd, amd, ["F4FN", "F4LN"], base_lists, name_changes=True)

******
['Suffix.Name', 'Race', 'Gender', 'Middle.Initial', 'Birth.Year', 'Appointed.Date', 'Last.Name', 'First.Name']
31914
******
******
['Suffix.Name', 'Race', 'Gender', 'Middle.Initial', 'Birth.Year', 'Appointed.Date', 'F4LN', 'First.Name']
3
******
******
['Suffix.Name', 'Race', 'Gender', 'Middle.Initial', 'Appointed.Date', 'Last.Name', 'First.Name']
1
******
******
['Suffix.Name', 'Gender', 'Middle.Initial', 'Birth.Year', 'Appointed.Date', 'Last.Name', 'First.Name']
1
******
******
['Suffix.Name', 'Race', 'Gender', 'Middle.Initial', 'Birth.Year', 'Appointed.Date', 'First.Name']
8
******
31927 82 1
      all_members_ID First.Name Last.Name Middle.Initial Suffix.Name  \
1942            1943     NACALA       BEY              M         NaN   

     Appointed.Date  Birth.Year  Gender   Race  F4FN F4LN  
1942     1989-12-26        1964  FEMALE  BLACK  NACA  BEY  


In [242]:
def StarMerge(df1, df2, on_list, keep_columns, return_unmerged = True):
    dfm = pd.DataFrame(columns = keep_columns + ['Match'])
    df2 = df2.loc[(df2["Star"].notnull()) & (df2["Star"] > 0)]
    stars = ["Star" + str(i) for i in range(1,11)]
    for mc_cols in on_list:
        for star in stars:
            mc1 = mc_cols + [star]
            mc2 = mc_cols + ["Star"]
            df1t = RemoveDuplicates(df1.loc[(df1[star].notnull()) & (df1[star] > 0), keep_columns[:1] + mc1], mc1)
            df2t = RemoveDuplicates(df2[keep_columns[1:] + mc2], mc2)
            dfmt = df1t.merge(df2t, left_on = mc1, right_on = mc2, how='inner')
            if dfmt.shape[0] > 0:
                print('******')
                print(mc1)
                print(dfmt.shape[0])
                print('******')
                dfmt['Match'] = '-'.join(mc + [star + "/" + "Star"])
                dfm = dfm.append(dfmt[keep_columns + ['Match']].reset_index(drop=True))
                df1 = df1.loc[~df1[keep_columns[0]].isin(dfm[keep_columns[0]])]
                df2 = df2.loc[~df2[keep_columns[1]].isin(dfm[keep_columns[1]])]
    if return_unmerged:
        return (dfm.reset_index(drop=True), df1, df2)
    else:
        return dfm.reset_index(drop=True)